MultiRAT preprocessing
================
Joanes Grandjean

![rat art](../assets/img/rat_art.png)

# Purpose
In this section I submit jobs to my high performance computer to perform the preprocessing, confound_correction, and analysis steps as implemented in RABIES. 
This section relies on the fact that my environment allows for parallel job submission using the `qsub` command. 



In [36]:
# init variables
init_folder='/home/traaffneu/joagra/code/MultiRat'  # location of the codes
analysis_folder='/project/4180000.19/multiRat'      # location of the bids folder and results
df_path='../assets/table/meta_data_20210622.tsv'    # meta-data table

scratch_folder='/data/joagra' # this is where preprocessed data will be temporarily stored
rabies_img = '/opt/rabies/0.3.5/rabies-0.3.5.simg' # location of the RABIES image

In [40]:
! mkdir -p $analysis_folder/export/qa/epi2anat
! mkdir -p $analysis_folder/export/qa/anat2template
! mkdir -p $analysis_folder/export/qa/template2std
! mkdir -p $analysis_folder/export/qa/temporal
! mkdir -p $analysis_folder/export/snr/atlas
! mkdir -p $analysis_folder/export/snr/tsnr
! mkdir -p $analysis_folder/export/motion/confound
! mkdir -p $analysis_folder/export/motion/FD
! mkdir -p $analysis_folder/export/log
! mkdir -p $analysis_folder/export/stim
! mkdir -p $analysis_folder/export/aromas_scan
! mkdir -p $analysis_folder/export/aromas_qa

In [37]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv(df_path, sep='\t')  # load the table

In [ ]:
from subprocess import call


qsub_call = "qsub -l 'walltime=00:01:00,mem=1gb, procs=1' "  # define how much time/memory/core to allocate per job

module_call = "module load singularity; module unload freesurfer; module load ANTs; module unload ANTs"  #the freesurfer and ants modules tricks as specific to my system due to some env contamination bugs 

template_args = "template=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Template.nii; mask=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Mask.nii; wm=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_WM_bin.nii.gz; csf=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_CSF_bin.nii.gz; atlas=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Brain_Atlases/SIGMA_Anatomical_Atlas/SIGMA_Anatomical_Brain_Atlas_rs.nii; roi=/template/roi/"

rabies_runmode_args =       "-p MultiProc "
rabies_preprocess_args =    "preprocess /rabies_in /rabies_out \
                            --anat_template ${template} \
                            --brain_mask ${mask} \
                            --WM_mask ${wm} \
                            --CSF_mask ${csf} \
                            --vascular_mask ${csf} \
                            --labels ${atlas} \
                            --TR ${TR}s \
                            --fast_commonspace \
                            --commonspace_resampling 0.3x0.3x0.3 \
                            --anat_inho_cor_method no_reg \
                            --bold_inho_cor_method no_reg \
                            --coreg_script Rigid "

rabies_confound_args =      "confound_correction /rabies_out /confound_out \
                            --read_datasink \
                            --highpass 0.01 \
                            --smoothing_filter 0.5 "
rabies_analysis_args =      "analysis /confound_out /analysis_out \
                            --seed_list"


In [11]:
with open ('test_job.sh', 'w') as rsh:
    rsh.writelines('mkdir test \n')

In [29]:
call(qsub_call + "${PWD}/test_job.sh", shell=True)

0

In [27]:
print(qsub_call + "${PWD}/test_job.sh")

qsub -l 'walltime=00:01:00,mem=1gb, procs=1' ${PWD}/test_job.sh
